In [76]:
from math import log
import operator

In [77]:
def calcShannonEnt(dataSet): #给定数据
    numEntries = len(dataSet) #获取数据长度
    labelCounts = {} #建立一个字典
    for featVec in dataSet: #循环数据
        currentLabel = featVec[-1] #取最后一个（倒数第一个）
        if currentLabel not in labelCounts.keys(): #判断当前键值是否存在于字典
            labelCounts[currentLabel] = 0 #不等于，则设置为0
        labelCounts[currentLabel] += 1 #等于，则设置为1
    shannonEnt = 0.0
    for key in labelCounts: #循环
        prob = float(labelCounts[key]) / numEntries #根据类别的次数，计算出类别的概率
        shannonEnt -= prob * log(prob, 2) #求熵，以 2 为底
    return shannonEnt #返回熵

In [78]:
def createDataSet():
    dataSet = [[1, 1, 'yes'],
              [1, 1, 'yes'],
              [1, 0, 'no'],
              [0, 1, 'no'],
              [0, 1, 'no']]
    labels = ['no surfacing', 'flippers' ]
    return dataSet, labels

In [79]:
myDat, labels = createDataSet()
myDat

[[1, 1, 'yes'], [1, 1, 'yes'], [1, 0, 'no'], [0, 1, 'no'], [0, 1, 'no']]

In [80]:
labels

['no surfacing', 'flippers']

In [81]:
calcShannonEnt(myDat)

0.9709505944546686

In [82]:
myDat[0][-1] = 'maybe'
calcShannonEnt(myDat)

1.3709505944546687

In [83]:
myDat

[[1, 1, 'maybe'], [1, 1, 'yes'], [1, 0, 'no'], [0, 1, 'no'], [0, 1, 'no']]

In [84]:
def splitDataSet(dataSet, axis, value):#待划分的数据集、特征、特征返回值
    retDataSet = [] #创建list
    for featVec in dataSet: #循环
        if featVec[axis] == value: #按某个特征划分数据集，当有符合要求的元素时，抽取取出来
            reducedFeatVec = featVec[:axis]
            reducedFeatVec.extend(featVec[axis+1:])#抽取(扩展，会把所有数据都放到后面)
            retDataSet.append(reducedFeatVec) #追加到后面（追加，会把追加的东西看成一个整体）
    return retDataSet #返回划分的数据

In [85]:
myDat, labels = createDataSet()
myDat

[[1, 1, 'yes'], [1, 1, 'yes'], [1, 0, 'no'], [0, 1, 'no'], [0, 1, 'no']]

In [86]:
splitDataSet(myDat, 0, 1)#第0位数据为1的数据提取出来

[[1, 'yes'], [1, 'yes'], [0, 'no']]

In [87]:
splitDataSet(myDat, 0 , 0)#第0位数据为0的数据提取出来

[[1, 'no'], [1, 'no']]

In [88]:
def chooseBestFeatureToSplit(dataSet):#选择最好的数据集划分方式
    numFeatures = len(dataSet[0]) - 1#获取特征长度
    baseEntropy = calcShannonEnt(dataSet)
    bestInfoGain = 0.0;
    baseFeature = -1;
    for i in range(numFeatures):
        featList = [example[i] for example in dataSet]
        uniqueVals = set(featList)
        newEntropy = 0.0
        for value in uniqueVals:
            subDataSet = splitDataSet(dataSet, i, value)
            prob = len(subDataSet)/float(len(dataSet))
            newEntropy += prob * calcShannonEnt(subDataSet)
        infoGain = baseEntropy - newEntropy
        if (infoGain > bestInfoGain):
            bestInfoGain = infoGain
            bestFeature = i
        return bestFeature

In [89]:
myDat

[[1, 1, 'yes'], [1, 1, 'yes'], [1, 0, 'no'], [0, 1, 'no'], [0, 1, 'no']]

In [90]:
Feature = chooseBestFeatureToSplit(myDat)
Feature

0

In [91]:
myDat

[[1, 1, 'yes'], [1, 1, 'yes'], [1, 0, 'no'], [0, 1, 'no'], [0, 1, 'no']]

In [92]:
def majorityCnt(classList):
    classCount = {}
    for vote in classList:
        if vote not in classCount.keys():
            classCount[vote] = 0
        else:
            classCount[vote] += 1
    sortedClassCount = sorted(classCount.iteritems(), key=operator.itemgetter(1), reverse= True)
    return sortedClassCount[0][0]

In [93]:
def createTree(dataSet, labels):#两个输入参数：数据集和标签列表
    classList = [example[-1] for example in dataSet]
    if classList.count(classList[0]) == len(classList):
        return classList[0]#类别相同时，停止继续划分
    if len(dataSet[0]) == 1:
        return majorityCnt(classList)#返回次数最多的特征
    bestFeat = chooseBestFeatureToSplit(dataSet)
    bestFeatLabel = labels[bestFeat]
    myTree = {bestFeatLabel:{}}
    del(labels[bestFeat])
    featValues = [example[bestFeat] for example in dataSet]
    uniqueVals = set(featValues)#得到列表包含所有的属性值
    for value in uniqueVals:
        subLabels = labels[:]#复制了类标签
        myTree[bestFeatLabel][value] = createTree(splitDataSet(dataSet,bestFeat,value),subLabels)
    return myTree

In [94]:
myDat, labels = createDataSet()

In [95]:
myTree = createTree(myDat, labels)

In [96]:
myTree

{'no surfacing': {0: 'no', 1: {'flippers': {0: 'no', 1: 'yes'}}}}